# Image Data Association

In [19]:
# optional: allow Jupyter to "hot reload" the Python modules I wrote, to avoid restarting the kernel after every change
%load_ext autoreload
%autoreload 2

## Part A: Formulating the Dataset

Note: the cells below do NOT need to be rerun!

### Downloading the Entire Dataset

In [1]:
! mkdir ./Data

mkdir: ./Data: File exists


In [2]:
! git clone https://github.com/openMVG/SfM_quality_evaluation.git 

Cloning into 'SfM_quality_evaluation'...


remote: Enumerating objects: 237, done.
remote: Total 237 (delta 0), reused 0 (delta 0), pack-reused 237
Receiving objects: 100% (237/237), 254.31 MiB | 13.61 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Updating files: 100% (214/214), done.


### Aggregating the Images

We only care about the `images/` directories in the following subsets of the data:
- `fountain-P11`
- `Herz-Jesus-P8`
- `entry-P10`

In [4]:
import glob

In [15]:
BASE_DATA_PATH = "./SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008"
subset_names = ["entry-P10", "Herz-Jesus-P8", "fountain-P11"]
file_ext_pattern = "*.jpg"

In [18]:
all_img_paths = list()

for subset in subset_names:
    pattern_for_subset_images = "/".join(
        [BASE_DATA_PATH, subset, "images", file_ext_pattern]
    )
    all_img_paths.extend(glob.glob(pattern_for_subset_images))

print(all_img_paths)

['./SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/entry-P10/images/0006.jpg', './SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/entry-P10/images/0007.jpg', './SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/entry-P10/images/0005.jpg', './SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/entry-P10/images/0004.jpg', './SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/entry-P10/images/0000.jpg', './SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/entry-P10/images/0001.jpg', './SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/entry-P10/images/0003.jpg', './SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/entry-P10/images/0002.jpg', './SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/entry-P10/images/0009.jpg', './SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/entry-P10/images/0008.jpg', './SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/Herz-Jesus-P8/images/0006.

Sweet. Lastly, we can load all these images into memory for further processing:

In [ ]:
from util import ops

In [ ]:
all_img_arrays = 